In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transforms.Normalize: output[channel] = (input[channel] - mean[channel]) / std[channel]
# This choice takes pixel values in [0,1] and shifts them to [-1,1]
batch_size = 10

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [2]:
# copy and pastse nn architecture 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, stride = 1, kernel_size = 5)   # six maps of 28x28
        self.pool = nn.MaxPool2d(2, 2)                              # six maps of 14x14
        self.conv2 = nn.Conv2d(6, 16, stride = 1, kernel_size = 5)  # sixteen maps of 10X10
        self.fc1 = nn.Linear(16 * 5 * 5, 120)                       # After (2,2) Pool2d on Conv2d 16*5*5                    
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# change path to match previous iteration
PATH = './cifar_net2.pth'
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [4]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 56 %
